In [1]:
import operator
import pandas
import pickle
import numpy
import datetime
import math
import os
import scipy
import scipy.optimize

In [22]:
def get_error(expected_pattern, simulated_pattern):
        ''' This method calculates the error between two activity patterns.
            Each bin in the expected activity pattern is compared to the
            simulated activity pattern and the error is calculated. The method
            returns the mean error over all the bins in the patterns. Two types
            of error calculation are supported: the Mean Abolute Error (MEA)
            and the relative MAE. The MAE of a bin is defined as the difference
            between the simulated and real value.  The relative error of a bin
            is defined as the difference between the simulated and real value
            the bin divided by the maximum error of this bin. The maximum error
            is the maximum distance of the real bin to either 0 or 1. The error
            is then (in both cases) multiplied by 100 to get the error range
            between 0 and 100.

        Args:
            expected_pattern (List[float]): The expected activity pattern.
            simulated_pattern (List[float]): The simulated activity pattern.

        Kwargs:
            method (str): Method of validation. 'MAE' for Mean Absolute Error
                or 'relMAE' for relative Mean Absolute Error. Default 'relMAE'.

        Returns:
            (float): The error between the expected and the simulated pattern.
        '''

        nr_bins = len(expected_pattern)
        error = 0
        for i in range(nr_bins):
            expected = expected_pattern[i]
            simulated = simulated_pattern[i]
            difference = numpy.abs(simulated - expected)
            max_distance = max(1 - expected, expected)
            error += 100 * ( difference / max_distance)


        return error / nr_bins

def transform_dist_version_x(original_dist, transforming_dist, mean_target, mean_origin, difference_factors):
    dist = [max(0, transform_val_version_x(original_dist[i], mean_target[i], mean_origin[i], transforming_dist[i],
                                           difference_factors))
            for i in range(len(original_dist))]
    return dist / sum(dist)


def transform_val_version_x(original_value, mean_target_value, mean_origin_value, difference_value, f):
    dev_origin = original_value - mean_origin_value
    dev_target = mean_target_value - original_value
    return original_value + f[0] * difference_value + f[1] * dev_origin + f[2] * dev_target

def get_optimal_transform_factor_version_x(original_dist, transform_dist, target_dist):
    best_score = 0.0
    best_combination = [0, 0, 0]
    history_scores = []
    history_best_cobinations = []
    size = len(transform_dist)
    max_factor = MAX_FACTOR
    min_factor = MIN_FACTOR
    step_size = STEP_SIZE
    temp = datetime.datetime.now()
    agents = list(data_agents)[:NUMBER_OF_AGENTS]
    dc_agents = {}
    score_counter = 0
    for agent in agents:
        dc_agent = sum_distributions(data_agents[agent]['disconnection_duration_dists'])
        dc_agent = dc_agent[:size] / sum(dc_agent[:size])
        dc_agents[agent] = list(dc_agent)
    for f0 in numpy.arange(min_factor, max_factor, step_size):
        print(f0, datetime.datetime.now() - temp)
        temp = datetime.datetime.now()
        for f1 in numpy.arange(min_factor, max_factor, step_size):
            for f2 in numpy.arange(min_factor, max_factor, step_size):
                factors_score = []
                for agent in agents:
                    pred = transform_dist_version_x(dc_agents[agent][:size], transform_dist[:size],
                                target_dist[:size], original_dist[:size], difference_factors = [f0, f1, f2])
                    naive_error = get_error(dc_agents[agent][:size], target_dist[:size])
                    target_error = get_error(pred, target_dist[:size])
                    score = naive_error - target_error
                    factors_score.append(score)
                score = numpy.mean(factors_score)
                if (score > best_score):
                    score_counter +=1
                    if score_counter > 10:
                        print('new best score (%.3f) for parameters %s' %(score, [f0, f1, f2]))
                    best_score = score
                    best_combination = [f0, f1, f2]
                if score > 0.01:
                    history_scores.append(score)
                    history_best_cobinations.append([f0, f1, f2])

    return best_combination, best_score, history_best_cobinations, history_scores

def sum_timedeltas(x):
    sum_ = datetime.timedelta(minutes = 0)
    for item in x:
        sum_ = operator.add(item, sum_)
    return sum_

def sum_distributions(distributions):
    total_distribution = pandas.Series()
    count = 0
    for dist in distributions:
        if isinstance(dist, pandas.DataFrame):
            continue
        if total_distribution.empty:
            count +=1
            total_distribution = dist
        else:
            count+=1
            total_distribution = total_distribution.radd(dist, level=None, fill_value=0, axis=0)
    return total_distribution.astype(int)

def load_agent(directory, agent_ID):
    with open(directory + agent_ID + '.pkl', 'rb') as agent_file:
        agent_data = pickle.load(agent_file)
    return agent_data

def load_dict_agents_data(directory, skip_changing_users = True, check_regular = True):
    high_to_low_users = ['5D763B2DCBFD274859F43EE1456A1DD0B68F72A82E0A8564372C4BD46B95DAC8',
                        '8DD214E9F9611E7D01E662E940AC973BCC34CF45C3D43D3306DFD3F9AEB5075C',
                        '9A79FD1E5649B3FA3978C6B7E682571ED5570F5D1A35CDE4E80C6F30DFCFAF2B',
                        '09CAA3715AE1BC35FC152487BBD693275E97DE039207824D1063E5A7A0E2DD6E',
                        '144984E1475253AC45D743910C4CF344A2D315EA12ACD21AB58971105DEAF522',
                        'B8256507B35A603D43CF7437CCC5712F33F1F54B82EB416FEA942725AAD3831C',
                        'F32DBA25D91138963F85144783F71A16E4C2CF7D1523F0644A6404B4C2F227C4',
                        'F9541CDC30F3E36CEE3EE6954D0AC7D7925C5539D5E751E79AD13734E455ADBC']
    low_to_high_users = ['6D47DA110FAA422E77B8DA82975F644315AC05C3D0E38B12A80816793452D857',
                        '8C45CDFC57F283C9BFB268CBD39725923FF8D22BCDF75E3E4FD054B079872459',
                        '9C510710140B57F6EB58CC760CCAE70064D161D0B993741CC479FAF65175B2CB',
                        '9D2D9AD61D9BCBF8A3CBBBEE6A031A7152EF2C28235E54CC5E1A478CC9BC9F73',
                        '17AFFF335DB8BF72259DD11C23D375ECA607613B2D9E1379AFF0922888A8EE90',
                        '53DDDDFF98D9CB22915B054A55F663BFDE941FFD8039A03248E559945C096031',
                        '80E9884E3E4B152B66E2B3A2861FE89BE1D48A346C16EF8739F9714F4BD6AEBB',
                        '606BD9C4F45EF4BD2710D2C4B178BF7356D2D9437BDF53B5593AB87F35A061B1',
                        '17897EE886276B2EA379234AC9112C0816C0C917977A8DB4E3282A3C9BE7D018',
                        '918130F5F1FC762D62308F082000F698F77C6CDBD34F9F4915835D484663A509',
                        'EDA3B229C2661C6E479A565173F553DB7A10275D5360E542DA626F96832C12C6',
                        'F346C8B51850F3D811CCDE0EC612153028C9F69E9B79C1FA9E4F5FA75F470AA9',
                        'FDC8791D2EE934F8A1DD22816E149DD2E37E185C985C772D38D44FE74AA207DE',
                        'F20D73D834CB2523C7E8E55643E2A7E6467DA5D4B4EBCA3547BC9FE49009E731']
    wth_are_u_doing_users = ['7DA773D56D9BABA6B82139F286FA593EB0685317075EE743F05A3DDE1A1A552A',
                          '62ABF94F3B61963B34B78EA7F062DE8681015F17730270CCAA8F6BD61828889F',
                          '70E685BA7C2BC723DA307881476A3E5FC3AD1FCABDDFBF3A5BBA69E8BBB53B94']
    result = {}
    all_agent_IDs = set([files[:-4] for files in os.listdir(directory) if files[0] != '.'])
    for ID in all_agent_IDs:
        if skip_changing_users and (ID in high_to_low_users or ID in low_to_high_users):
            continue
        agent_data = load_agent(directory, ID)
        if check_regular and agent_data['user_type'] != 'regulier':
            continue
        else:
            result[ID] = agent_data
    return result

def print_and_save_results(best_factors, best_score, history_factors, history_scores, filename):
    results = {}
    results['best_factors'] = best_factors
    results['best_score'] = best_score
    results['history_factors'] = history_factors
    results['history_scores'] = history_scores
    resulting_file = result_directory + filename + '.pkl'
    with open(resulting_file, 'wb') as f:
        pickle.dump(results, f)
    print('Results: factor = %s, score = %.5f' %(best_factors, best_score))
    print('Results saved in %s' %resulting_file)


In [4]:
NUMBER_OF_AGENTS = 300
MIN_FACTOR = -3
MAX_FACTOR = 3
STEP_SIZE = 0.5

In [5]:
data_agents = load_dict_agents_data('data/agent_database/all_non_changing_agents/')
directory = 'data/simulation_pkls/'
with open(directory + 'disconnection_distribution_phev.pkl', 'rb') as f:
    disconnection_distribution_phev = pickle.load(f)
with open(directory + 'disconnection_distribution_high_fev.pkl', 'rb') as f:
    disconnection_distribution_high_fev = pickle.load(f)
with open(directory + 'disconnection_distribution_low_fev.pkl', 'rb') as f:
    disconnection_distribution_low_fev = pickle.load(f)
with open(directory + 'connection_distribution_phev.pkl', 'rb') as f:
    connection_distribution_phev = pickle.load(f)
with open(directory + 'connection_distribution_high_fev.pkl', 'rb') as f:
    connection_distribution_high_fev = pickle.load(f)
with open(directory + 'connection_distribution_low_fev.pkl', 'rb') as f:
    connection_distribution_low_fev = pickle.load(f)
with open(directory + 'transformation_disconnection_phev_to_high.pkl', 'rb') as f:
    diff_dc_phev_to_high = pickle.load(f)
with open(directory + 'transformation_disconnection_phev_to_low.pkl', 'rb') as f:
    diff_dc_phev_to_low = pickle.load(f)
with open(directory + 'transformation_connection_phev_to_high.pkl', 'rb') as f:
    diff_con_phev_to_high = pickle.load(f)
with open(directory + 'transformation_connection_phev_to_low.pkl', 'rb') as f:
    diff_con_phev_to_low = pickle.load(f)


In [26]:
def get_score(x, dists, size, original_dist, transform_dist, target_dist):
    f0 = x[0]
    f1 = x[1]
    f2 = x[2]
    factors_score = []
    for dist in dists:
        this_size = min(size, len(dist))
        if agent_size == 0:
            print('empty dist')
            continue
        pred = transform_dist_version_x(dist[:this_size], transform_dist[:this_size],
                    target_dist[:agent_size], original_dist[:this_size], difference_factors = [f0, f1, f2])
        naive_error = get_error(dist[:this_size], target_dist[:this_size])
        target_error = get_error(pred, target_dist[:this_size])
        score = naive_error - target_error
        factors_score.append(score)
    score = numpy.mean(factors_score)
    return -1.0*score

In [36]:
dists_centers = []
for agent in agents:
    for center in data_agents[agent]['centers_info']:
        dist = sum_distributions(data_agents[agent]['connection_duration_dists'][center])
        dist = dist[:size] / sum(dist[:size])
        dists_centers.append(list(dist))

In [ ]:
'''diff_dc_phev_to_high 
[0.16259663, -0.16202726,  0.83797553]
-0.15294582787757854


'''

In [33]:
with open('test.pkl', 'wb') as f:
    pickle.dump(result, f)

In [34]:
with open('test.pkl', 'rb') as f:
    test = pickle.load(f)

In [29]:
agents = set(list(data_agents)[:20])
dc_agents = {}
original_dist = connection_distribution_phev
transform_dist = list(diff_con_phev_to_high)
target_dist = connection_distribution_high_fev
size = len(transform_dist)
for agent in agents:
    dc_agent = sum_distributions(data_agents[agent]['disconnection_duration_dists'])
    dc_agent = dc_agent[:size] / sum(dc_agent[:size])
    dc_agents[agent] = list(dc_agent)
print("starting basin hopping")
start = datetime.datetime.now()
result = scipy.optimize.basinhopping(get_score, [0,0,0],  niter=10, T=1.0, stepsize=0.5, 
    minimizer_kwargs = {"args": (agents, dc_agents, size, original_dist, transform_dist, target_dist)}, 
    take_step=None, accept_test=None, callback=None, interval=5, disp=True, niter_success=None)
print(datetime.datetime.now() - start)
print(result)

starting basin hopping
basinhopping step 0: f -0.186435
basinhopping step 1: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
found new global minimum on step 1 with function value -0.186435
basinhopping step 2: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
basinhopping step 3: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
basinhopping step 4: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
adaptive stepsize: acceptance rate 0.800000 target 0.500000 new stepsize 0.555556 old stepsize 0.5
basinhopping step 5: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
basinhopping step 6: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
basinhopping step 7: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
basinhopping step 8: f -0.186435 trial_f -0.186435 accepted 1  lowest_f -0.186435
found new global minimum on step 8 with function value -0.186435
basinhopping step 9: f -0.186435 trial_f -0.186435 accepted

In [ ]:
print('getting optimal disconnection phev to high factors')
result_directory = 'data/battery_size/'
best_factors, best_score, history_factors, history_scores = get_optimal_transform_factor_version_x(disconnection_distribution_phev,
    list(diff_dc_phev_to_high), disconnection_distribution_high_fev)
print('disconnection phev to high')
filename = 'phev_to_high_disconnection_transformation_factors_' + str(NUMBER_OF_AGENTS) + '_' + str(MIN_FACTOR) + '_' + str(MAX_FACTOR) + '_' + str(STEP_SIZE)
print_and_save_results(best_factors, best_score, history_factors, history_scores, filename)

# print('getting optimal disconnection phev to low factors')
# best_factors, best_score, history_factors, history_scores = get_optimal_transform_factor_version_x(disconnection_distribution_phev,
#     list(diff_dc_phev_to_low), disconnection_distribution_low_fev)
# print('disconnection phev to low')
# filename = 'phev_to_low_disconnection_transformation_factors_' + str(NUMBER_OF_AGENTS) + '_' + str(MIN_FACTOR) + '_' + str(MAX_FACTOR) + '_' + str(STEP_SIZE)
# print_and_save_results(best_factors, best_score, history_factors, history_scores, filename)

# print('getting optimal connection phev to high factors')
# best_factors, best_score, history_factors, history_scores = get_optimal_transform_factor_version_x(connection_distribution_phev,
#     list(diff_con_phev_to_high), connection_distribution_high_fev)
# print('connection phev to high')
# filename = 'phev_to_high_connection_transformation_factors_' + str(NUMBER_OF_AGENTS) + '_' + str(MIN_FACTOR) + '_' + str(MAX_FACTOR) + '_' + str(STEP_SIZE)
# print_and_save_results(best_factors, best_score, history_factors, history_scores, filename)

# print('getting optimal connection phev to low factors')
# best_factors, best_score, history_factors, history_scores = get_optimal_transform_factor_version_x(connection_distribution_phev,
#     list(diff_con_phev_to_low), connection_distribution_low_fev)
# print('connection phev to low')
# filename = 'phev_to_low_connection_transformation_factors_' + str(NUMBER_OF_AGENTS) + '_' + str(MIN_FACTOR) + '_' + str(MAX_FACTOR) + '_' + str(STEP_SIZE)
# print_and_save_results(best_factors, best_score, history_factors, history_scores, filename)
